In [7]:
import os 
from pathlib import Path
import sys
import subprocess
import time

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

def split_into_graphs_help(lines: list[str], graphs: list[list[str]]):
    for line in lines:
        if "detected" in line or "execution explored" in line or "wall-clock time" in line:
            return graphs
        elif line == "\n" or line == "":
            continue
        elif "Execution Graph" in line:
            graphs.append([])
        else:
            graphs[-1].append(line)
    return graphs

def merge_strs(xss: list[list[str]]) -> list[object]:
    res = []
    for xs in xss:
        res.append("\n".join(xs))
    return res
        
def split_into_graphs(lines: list[str]):
    return merge_strs(split_into_graphs_help(lines, []))

def get_genmc_path() -> Path:
    return Path('/root') / 'genmc-xmm' / 'genmc'

def list_test_paths() -> [Path]:
    tests_path = Path('/root') / 'xmm-benchmarks' / 'tests'
    assert os.path.isdir(tests_path)
    tests = os.listdir(tests_path)
    tests.sort()
    xs = [tests_path / test for test in tests]
    return [x for x in xs if os.path.isdir(x)]

def print_result(path, testname, execs, dups, expected, dirname):
    if execs == expected:
        print(bcolors.OKGREEN, f"PASSED ({execs} executions, {dups} duplicates)", path / testname, bcolors.ENDC)
    elif dirname in expected_fails:
        print(bcolors.OKBLUE, "EXPECTED FAILED", path / testname, "expected", expected, "got (", execs, "executions, ", dups, " duplicates)", bcolors.ENDC)
    else:
        print(bcolors.FAIL, "FAILED", path / testname, "expected", expected, "got (", execs, "executions, ", dups, " duplicates)", bcolors.ENDC)

def get_digits_after(pattern: str, string: str):
    index = string.index(pattern) 
    assert(index >= 0)
    index += len(pattern)

    num_part_len = 0
    for c in string[index:]:
        if c.isdigit():
            num_part_len += 1
        else:
            break
    
    assert(num_part_len > 0)
    num_str = string[index:index+num_part_len]
    return int(num_str)


def test_xmm(test_path: Path):
    xmm_expected_file = test_path / "expected.xmm.in"
    if not os.path.isfile(xmm_expected_file):
        print("expected.xmm.in not found for ", test_path)
        return

    print("")
    dirname = test_path.name
    print(dirname)
    
    xmm_expected = None
    with open(xmm_expected_file) as f:
        xmm_expected_str = f.read()
        try:
            xmm_expected = int(xmm_expected_str)
        except ValueError:
            print(f"could not convert contents of {xmm_expected_file}: {xmm_expected_str} to int value")
            exit(1)
    
    variants_path = test_path / "variants"
    for variant in os.listdir(variants_path):
        genmc_result = subprocess.run([genmc_path, "--print-exec-graphs", "--XMM", variants_path / variant], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        genmc_output = genmc_result.stdout.decode()
        genmc_output_stderr = genmc_result.stderr.decode()

        if genmc_result.returncode != 0:
            print("genmc exited with exit code ", genmc_result.returncode)
            print(variants_path / variant)
            print(genmc_output)
            print(genmc_output_stderr)
            exit(1)
        
        execs = get_digits_after("Number of complete executions explored: ", genmc_output)
        dups = get_digits_after("Number of duplicate executions explored: ", genmc_output)
        print_result(variants_path, variant, execs, dups, xmm_expected, dirname)
        if dirname in expected_fails:
            continue

        combined_output = genmc_output_stderr + genmc_output
        variantname = variant.split('.')[0]
        with open(test_path / f"expected.xmm.output.{variantname}", "r") as f:
            contents = f.read()
            actual = set(split_into_graphs(combined_output.split("\n")))
            expected = set(split_into_graphs(contents.split("\n")))
            for e in expected:
                if e not in actual:
                    print(bcolors.FAIL, "FAILED", variants_path / variant, "missing graph\n", e, bcolors.ENDC)
            for a in actual:
                if a not in expected:
                    print(bcolors.FAIL, "FAILED", variants_path / variant, "unexpected graph\n", a, bcolors.ENDC)

genmc_path = get_genmc_path()
test_paths = list_test_paths()
for path in test_paths:
    assert os.path.isdir(path)

expected_fails = set(["LB+coh-cyc", "LB+coh-cyc+Wd", "LB+porf-suffix"])

argv = sys.argv
start_time = time.time()
for path in test_paths:
    test_xmm(path)
delta_t = time.time() - start_time
print(f"Total execution time: {delta_t:.3f}s")


LB
 PASSED (4 executions, 2 duplicates) /root/xmm-benchmarks/tests/LB/variants/lb0.c 

LB+CoRR
 PASSED (14 executions, 2 duplicates) /root/xmm-benchmarks/tests/LB+CoRR/variants/lb+corr0.c 

LB+CoRRx
 PASSED (22 executions, 2 duplicates) /root/xmm-benchmarks/tests/LB+CoRRx/variants/lb+corrx0.c 

LB+LB
 PASSED (9 executions, 10 duplicates) /root/xmm-benchmarks/tests/LB+LB/variants/lb+lb0.c 

LB+LB+if
 PASSED (3 executions, 0 duplicates) /root/xmm-benchmarks/tests/LB+LB+if/variants/lb+lb+if0.c 

LB+LB+if-reversed
 PASSED (5 executions, 1 duplicates) /root/xmm-benchmarks/tests/LB+LB+if-reversed/variants/lb+lb+if0.c 

LB+RR-same-loc
 PASSED (6 executions, 0 duplicates) /root/xmm-benchmarks/tests/LB+RR-same-loc/variants/main0.c 

LB+SB
 PASSED (16 executions, 23 duplicates) /root/xmm-benchmarks/tests/LB+SB/variants/lb+sb0.c 

LB+W
 PASSED (12 executions, 4 duplicates) /root/xmm-benchmarks/tests/LB+W/variants/lb+w0.c 
 PASSED (12 executions, 4 duplicates) /root/xmm-benchmarks/tests/LB+W/vari